In [ ]:
# default_exp analysis

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# analysis

> Some functions for doing analyses on the alberta covid data

In [ ]:
#export
import pandas as pd
import numpy as np

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def calculate_doublingtimes(df:pd.DataFrame, col_suffix:str='cumCases',
                            median_incubation_period:float=5.2, combine_df=True):
    '''
    given a dataframe look through and calculate the doubling times. Both doubling time based
    on the first occurance of covid-19 and doubling time based on a rolling window which
    looks back as far as the median incubation period (converted to integer using ceiling)

    Parameters:
    
        df: pd.DataFrame
            the dataframe containing the region data
        col_suffix:str
            cumulative column suffix used to filter the dataframe to only grab cumulative data
            uses pd.DataFrame.filter(like=col_suffix)
        combine_df:bool
            if True will return a new dataframe with the doubling time data joined with the old dataframe
            if False will only return a dataframe with the doubling time data
    
    ------
    Return:
    
        regionDF_doublingtime: DataFrame
    '''
    median_incub_prd = np.ceil(median_incubation_period)
    filtered_df = df.filter(like=col_suffix)
    doubling_time = dict()
    for label, cum_data in filtered_df.iteritems():
        start_idx = cum_data.to_numpy().nonzero()[0][0] # assumes date is used as df index
        min_idx_rw = start_idx + median_incub_prd
        start_cases = cum_data.iloc[start_idx]
        start_date = cum_data.index[start_idx]

        dt_values = [0]
        dt_key = label.replace(col_suffix, 'dtime')
        dt_rw_values = [0]
        dt_rw_key = label.replace(col_suffix, 'dtime_rw')

        for idx in range(1, len(cum_data)):
            new_cases = cum_data.iloc[idx]
            new_date = cum_data.index[idx]

            # calculate the doubling time
            t_delta = new_date - start_date
            c_delta = new_cases / start_cases

            dtime = 0 # default
            if c_delta > 1: # log(1) = 0, log(x) with x: 0 < x > 1 is equal to a negative number
                dtime = (t_delta.days * np.log(2))/(np.log(c_delta))
            dt_values.append(dtime)

            # Rolling window calculation
            dtime_rw = 0 # default
            if idx >= min_idx_rw:
                idx_rw = idx - 6
                old_cases_rw = cum_data.iloc[idx_rw]
                old_date_rw = cum_data.index[idx_rw]
                # calculate the doubling time
                t_delta_rw = new_date - old_date_rw
                c_delta_rw = new_cases / old_cases_rw
                if c_delta_rw > 1: # log(1) = 0, log(x) with x: 0 < x > 1 is equal to a negative number
                    dtime_rw = (t_delta_rw.days * np.log(2))/(np.log(c_delta_rw))
            dt_rw_values.append(dtime_rw)

        doubling_time[dt_key] = dt_values
        doubling_time[dt_rw_key] = dt_rw_values
    df_dt = pd.DataFrame(doubling_time, index=filtered_df.index)
    if combine_df:
        return df.join(df_dt)
    return df_dt

In [ ]:
show_doc(calculate_doublingtimes)

<h4 id="calculate_doublingtimes" class="doc_header"><code>calculate_doublingtimes</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>calculate_doublingtimes</code>(**`df`**:`DataFrame`, **`col_suffix`**:`str`=*`'cumCases'`*, **`median_incubation_period`**:`float`=*`5.2`*, **`combine_df`**=*`True`*)

given a dataframe look through and calculate the doubling times. Both doubling time based
on the first occurance of covid-19 and doubling time based on a rolling window which
looks back as far as the median incubation period (converted to integer using ceiling)

Parameters:

    df: pd.DataFrame
        the dataframe containing the region data
    col_suffix:str
        cumulative column suffix used to filter the dataframe to only grab cumulative data
        uses pd.DataFrame.filter(like=col_suffix)
    combine_df:bool
        if True will return a new dataframe with the doubling time data joined with the old dataframe
        if False will only return a dataframe with the doubling time data

------
Return:

    regionDF_doublingtime: DataFrame

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_webscraper.ipynb.
Converted 01_analysis.ipynb.
Converted index.ipynb.
